In [ ]:
pip install deep-translator

In [ ]:
from deep_translator import GoogleTranslator


def back_translate(texts, language_src, language_dst):
  """Implements back translation"""
  # Translate from source to target language
  translated = GoogleTranslator(source='auto', target=language_dst).translate(texts)

  # Translate from target language back to source language
  back_translated = GoogleTranslator(source='auto', target=language_src).translate(translated)

  return back_translated

In [ ]:
import pandas as pd

df_weak_sel_cleaned= pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/data_for_augmentation_n_min_max_idoneidade_financeira.csv')
df_weak_sel_cleaned

In [ ]:
import tqdm.auto as tqdm
from unidecode import unidecode
import re
df_translated=pd.DataFrame(columns=df_weak_sel_cleaned.columns)
for i, row in tqdm.tqdm(df_weak_sel_cleaned.iterrows(), total=df_weak_sel_cleaned.shape[0]):
    df_translated.loc[i]=row
    if len(df_translated.loc[i,'text'])<5000:
        result=back_translate(df_translated.loc[i,'text'],'pt','ja')
    else:
        result=""
        sentences = re.split(r'\.\s+', df_translated.loc[i,'text'])
        for sentence in tqdm.tqdm(sentences,total=len(sentences)):
            if len(result)==0:
                result=back_translate(sentence,'pt','ja')
            else:
                result=result+". "+back_translate(sentence,'pt','ja')
    result = re.sub(r'\.+', '.', result)
    result = unidecode(result)
    result = result.lower()
    df_translated.loc[i,'text']=result

In [ ]:
df_weak_sel_cleaned.loc[0,'text']

In [ ]:
df_translated.loc[0,'text']

In [ ]:
df_translated.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/Indicios/DatasetsWeak/DataAugmentation/data_back_translated_n_min_max_idoneidade_financeira.csv')

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Helper function to download data for a language
def download(model_name):
  tokenizer = MarianTokenizer.from_pretrained(model_name)
  model = MarianMTModel.from_pretrained(model_name).to('cpu')
  return tokenizer, model

# download model for English -> Romance
src_lang_tokenizer, src_lang_model = download('Helsinki-NLP/opus-mt-en-ROMANCE')
# download model for Romance -> English
tmp_lang_tokenizer, tmp_lang_model = download('Helsinki-NLP/opus-mt-ROMANCE-en')

In [ ]:
def translate(texts, model, tokenizer, language):
  """Translate texts into a target language"""
  # Format the text as expected by the model
  formatter_fn = lambda txt: f"{txt}" if language == "en" else f">>{language}<< {txt}"
  original_texts = [formatter_fn(txt) for txt in texts]
  # Tokenize (text to tokens)
  input_ids = tokenizer(original_texts, return_tensors="pt",padding=True,truncation=True).input_ids.to('cpu')
  # Translate
  translated = model.generate(input_ids)

  # Decode (tokens to text)
  translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)

  return translated_texts

def back_translate(texts, language_src, language_dst):
  """Implements back translation"""
  # Translate from source to target language
  translated = translate(texts, tmp_lang_model, tmp_lang_tokenizer, language_dst)

  # Translate from target language back to source language
  back_translated = translate(translated, src_lang_model, src_lang_tokenizer, language_src)

  return back_translated

In [ ]:
import tqdm.auto as tqdm
import re
df_translated=pd.DataFrame(columns=df_weak_sel_cleaned.columns)
for i, row in tqdm.tqdm(df_weak_sel_cleaned.iterrows(), total=df_weak_sel_cleaned.shape[0]):
    df_translated.loc[i]=row
    sentences = re.split(r'\.\s+', df_translated.loc[i,'text'])
    result=[]
    for sentence in tqdm.tqdm(sentences,total=len(sentences)):
        result.append(back_translate(sentence,'pt_BR','en'))
    df_translated.loc[i,'text']=". ".join(result)


In [ ]:
import random

def insert_red_flag(amostra,texto):
    # Split the text into sentences
    sentences_list = re.split('\. |; |: |, |\*|\n', texto)

    # Insert at a random position
    random_index = random.randint(0, len(sentences_list))
    sentences_list.insert(random_index, amostra)

    # Rejoin the list of sentences
    return '. '.join(sentences_list)


In [ ]:
import re
def find_sentence_with_substring(text, regex):    
    
    sentences = re.split('\. |; |, |\*|\n', text)
    sents = []
    for sentence in sentences:
        if re.search(regex, sentence):
            sents.append(sentence)
    return sents

In [ ]:
metade_dataset=int(df_weak_sel_cleaned.shape[0]/2)
cols_indicios=df_weak_sel_cleaned.columns.values.tolist()[1:]
quantos_aumentar={}
for col in cols_indicios:
    quantos_aumentar[col]=metade_dataset-df_weak_sel_cleaned.value_counts(col).values[1]

In [ ]:
quantos_aumentar

In [ ]:
exp_certidao_protesto = re.compile(r'((certid[aã]o).{0,100}(protesto))',flags=re.IGNORECASE|re.S)
exp_integralizado = re.compile(r'((capital.{0,20}integralizado)|(patrim[oô]nio.{0,20}integralizado))', flags=re.IGNORECASE|re.S)
exp_idoneidade_financeira = re.compile('((atesta[do]*)|(certid[ãa]o)|(declara[çc]*[ãa]*[o]*))(.{0,70}((i[ni]*doneidade)).{0,20}((financeira)|(banc[áa]ria)))', flags=re.IGNORECASE|re.S)
exp_n_min_max_limitacao_atestados = re.compile(r'((((dois|duas)|(tr[êe]s)|(quatro)|(cinco)).{0,10}((atestado[s]?)|(certid[aãoões]*))).{0,20}((capacidade t[eé]cnica)|(qualifica[cç][aã]o t[eé]cnica)))', flags=re.IGNORECASE|re.S)
exp_certificado_boas_praticas = re.compile(r'(((certificado[s]?)|(atestado[s]?)|(certid[aã]o)).{0,20}(boa[s]?.{0,5}pr[aá]tica[s]?))', flags=re.IGNORECASE|re.S)
exp_licenca_ambiental = re.compile(r'((licen[cç]a).{0,20}(ambiental))', flags=re.IGNORECASE|re.S)
exp_comprovante_localizacao = re.compile('((alvar[aá])|(comprov[mnteçcãao]*)).{0,20}((localiza[çc][ãa]o)|(funcionamento))',flags=re.IGNORECASE|re.S)
expressoes={'certidao_protesto':exp_certidao_protesto,'integralizado':exp_integralizado,'idoneidade_financeira':exp_idoneidade_financeira,
            'n_min_max_limitacao_atestados':exp_n_min_max_limitacao_atestados,'certificado_boas_praticas':exp_certificado_boas_praticas,
            'licenca_ambiental':exp_licenca_ambiental,'comprovante_localizacao':exp_comprovante_localizacao}

In [ ]:
sentences_list={'certidao_protesto':[],'integralizado':[],
                'idoneidade_financeira':[],'n_min_max_limitacao_atestados':[],
                'certificado_boas_praticas':[],'licenca_ambiental':[],'comprovante_localizacao':[]}

In [ ]:
for i,row in df_weak_sel_cleaned.iterrows():
    for col in cols_indicios:
        if row[col]==1:
            sentences_list[col]=list(set(sentences_list[col]+find_sentence_with_substring(row['text'], expressoes[col])))

In [ ]:
for col in cols_indicios:
    print(col,len(sentences_translated[col]))

In [ ]:
import pickle

file_path = "sentences_translated.pkl"
with open(file_path, "rb") as file:
    sentences_translated = pickle.load(file)


In [ ]:
#sentences_translated={}
#for col in cols_indicios:
#    sentences_translated[col]=back_translate(sentences_list[col],"pt_BR","en")

In [ ]:
for col in cols_indicios:
    falta=quantos_aumentar[col]
    df_syntetic_insert=df_weak_sel_cleaned[df_weak_sel_cleaned[col]==0].sample(falta)
    index_list = df_syntetic_insert.index.tolist()
    while falta>0:
        random.shuffle(index_list)
        ind=index_list.pop()
        texto=df_syntetic_insert.loc[ind,'text']
        df_weak_sel_cleaned.loc[ind,'text']=insert_red_flag(sentences_translated[col][random.randint(0,len(sentences_translated[col])-1)],texto)
        df_weak_sel_cleaned.loc[ind,col]=1
        falta-=1

In [ ]:
for col in cols_indicios:
    print(col,df_weak_sel_cleaned.value_counts(col).values)

In [ ]:
df_weak_sel_cleaned.to_csv('./train-BERT-indicios-weak/dataset_weak_augmented_insert_red_flags_translated.csv',index=False)